In [8]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage
from skimage.measure import label
from tqdm import tqdm
import re
from multiprocess import Pool
import os
import warnings
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=RuntimeWarning)

In [9]:
ds_prcp = xr.open_dataset(f"/scratch/nf33/hk25_LSP/Germany/Darwin/pr_hourly.nc")
domain_area = 200.0*200.0*25 

In [10]:
prcp = ds_prcp["pr"]*3600


0.5488934655786771 mm/hr

In [11]:
num_obj_arr = np.zeros(len(ds_prcp.time))
    
## array for total area
tot_area_arr = np.zeros(len(ds_prcp.time))
## array to store mean cell size
mean_obj_area_arr = np.zeros(len(ds_prcp.time))
## array to store convective area fraction
area_frac_arr = np.zeros(len(ds_prcp.time))
## mean precipitation over convective area
cvt_mean_prcp_arr = np.zeros(len(ds_prcp.time))
## mean precipitation over the entire radar scan
tot_mean_prcp_arr = np.zeros(len(ds_prcp.time))
## total convective precipitation
cvt_tot_prcp_arr = np.zeros(len(ds_prcp.time))

In [12]:
pr_th = 0.5488934655786771

In [13]:
for its in range(0,len(ds_prcp.time)):
    # data = ds_prcp["steiner"].isel(time=its)
    prcp = ds_prcp["pr"].isel(time=its)*3600
    cvt_prcp = np.sum(prcp.values[prcp.values>=pr_th])    
    cv_obj = prcp.copy().fillna(0)
    cv_obj.values[cv_obj.values < pr_th] = 0
    cv_obj.values[cv_obj.values >= pr_th] = 6
    ## use scikit learn to label
    label_arr = label(cv_obj)
    ## find unique objects
    unique_label = np.unique(label_arr)
    ## get the number of objects 
    num_obj_arr[its] = len(unique_label) - 1   ## the background is 0
    ## individual object area
    ind_obj_area = np.zeros(len(unique_label) - 1)
    for i in unique_label:
        if i == 0:
            continue
        else:
            ind_obj_area[i-1]= np.sum(label_arr == i)*25 ## the data is 5 km by 5 km so this is 25 km**2
            ## this may be needed for precip thresholds
            # if obj_area <=5:
            #     label_new[label_new==i] = 0
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore", category=RuntimeWarning)
            
            tot_area_arr[its] = np.sum(ind_obj_area)
            mean_obj_area_arr[its] = tot_area_arr[its]/num_obj_arr[its]
            area_frac_arr[its] = tot_area_arr[its]/domain_area
        
            cvt_mean_prcp_arr[its] = cvt_prcp/tot_area_arr[its]
            tot_mean_prcp_arr[its] = cvt_prcp/domain_area
            cvt_tot_prcp_arr[its] = cvt_prcp

In [14]:
## save to netcdf
out = xr.Dataset(
    {
        "num_obj": (("time"), num_obj_arr),
        "tot_area":  (("time"), tot_area_arr),
        "mean_obj_area":  (("time"), mean_obj_area_arr),
        "area_frac":  (("time"), area_frac_arr),
        "cvt_mean_prcp":  (("time"), cvt_mean_prcp_arr),
        "tot_mean_prcp":  (("time"), tot_mean_prcp_arr),
        "cvt_tot_prcp":  (("time"), cvt_tot_prcp_arr),
        
    },
    coords={
        "time":  ds_prcp.time.values,
    },
)
out.to_netcdf(f"/scratch/nf33/hk25_LSP/Germany/Darwin/number_size_hourly.nc")
out.close()
ds_prcp.close()